<a href="https://colab.research.google.com/github/avkaz/DeepLearningPetIdentification/blob/fine-tuned-v0/fine101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

import requests
# Correct raw URL for the utility.py file
url = "https://raw.githubusercontent.com/avkaz/DeepLearningPetIdentification/main/utility.py"

# Fetch and save the file locally
response = requests.get(url)
with open("utility.py", "wb") as f:
    f.write(response.content)



In [41]:
!pip install faiss-cpu
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import requests
import json
import pandas as pd
import faiss
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from google.colab import files
from utility import download_and_preprocess_image, get_data, download_file_from_google_drive, load_and_prepare_dataframe
from itertools import islice


In [36]:
filtered_data = {k: v for k, v in data.items() if len(v['images']) > 3}

In [37]:
len(filtered_data)

1815

In [38]:
first_100 = dict(islice(filtered_data.items(), 800))

filter out pets for training, only using postings that have 3+ pictures for triplet loss

In [39]:

def filter_pets_by_images(data, min_images=3):
    """Filters pets with at least 'min_images' images."""
    return {key: value for key, value in data.items() if len(value['images']) >= min_images}


def triplet_loss(margin=1.0):
    """Triplet Loss function."""
    def loss(y_true, y_pred):
        anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
        positive_distance = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        negative_distance = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        loss_value = tf.maximum(positive_distance - negative_distance + margin, 0.0)
        return tf.reduce_mean(loss_value)
    return loss



def create_model(input_dim):
    """Create a model that processes embeddings directly (e.g., from EfficientNetB0)."""
    inputs = tf.keras.Input(shape=(input_dim,))  # Input layer for the flattened embedding vector
    x = layers.Dense(256, activation='relu')(inputs)  # Dense layer to process embeddings
    x = layers.Dense(128, activation='relu')(x)  # Another dense layer for more abstraction
    output = layers.Dense(1, activation='sigmoid')(x)  # Output layer (adjust if needed)
    model = Model(inputs, output)
    return model

def generate_embeddings(model, image_urls):
    """Generate embeddings for the list of image URLs."""
    embeddings = []
    for url in image_urls:
        img = download_and_preprocess_image(url, target_size=(128, 128))
        emb = model.predict(np.expand_dims(img, axis=0))
        embeddings.append(emb.flatten())
    return np.array(embeddings)

In [43]:
model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

retry

In [30]:
download_file_from_google_drive()

Downloading...
From (original): https://drive.google.com/uc?id=1VR5GWGrVjEtJHEzTPIB-EHDQMG3UnmZ9&export=download
From (redirected): https://drive.google.com/uc?id=1VR5GWGrVjEtJHEzTPIB-EHDQMG3UnmZ9&export=download&confirm=t&uuid=08027425-b16f-466c-b2d9-0e96d05be908
To: /content/pets_pair.json

  0%|          | 0.00/3.02G [00:00<?, ?B/s]
  0%|          | 2.10M/3.02G [00:00<02:23, 21.0MB/s]
  1%|          | 19.4M/3.02G [00:00<00:27, 109MB/s] 
  1%|          | 35.7M/3.02G [00:00<00:22, 131MB/s]
  2%|▏         | 55.1M/3.02G [00:00<00:19, 155MB/s]
  2%|▏         | 73.4M/3.02G [00:00<00:17, 165MB/s]
  3%|▎         | 90.7M/3.02G [00:00<00:17, 167MB/s]
  4%|▎         | 107M/3.02G [00:00<00:20, 142MB/s] 
  4%|▍         | 123M/3.02G [00:00<00:21, 133MB/s]
  5%|▍         | 137M/3.02G [00:00<00:21, 135MB/s]
  5%|▌         | 151M/3.02G [00:01<00:21, 134MB/s]
  5%|▌         | 165M/3.02G [00:01<00:24, 115MB/s]
  6%|▌         | 178M/3.02G [00:01<00:24, 117MB/s]
  6%|▋         | 196M/3.02G [00:01<00:21,

File saved to: ./pets_pair.json


In [44]:
embeddings_df = load_and_prepare_dataframe('./pets_pair.json')

Loading the JSON file into a pandas DataFrame...
Unwrapping image tensors from lists...
DataFrame after processing:
   plemeno1   vek1         barva1          velikost1  \
0  Kříženec  5 let          Černá  Střední - 10-17kg   
1  Kříženec  5 let          Černá  Střední - 10-17kg   
2  Kříženec  5 let          Černá  Střední - 10-17kg   
3  Kříženec  5 let          Černá  Střední - 10-17kg   
4  Jezevčík  5 let  Tmavý divočák  Střední - 10-17kg   

                                              fotka1  plemeno2      vek2  \
0  [[[0.14509804546833002, 0.15686275064945202, 0...  Kříženec     5 let   
1  [[[0.14509804546833002, 0.15686275064945202, 0...  Kříženec     5 let   
2  [[[0.14509804546833002, 0.15686275064945202, 0...  Kříženec    2 roky   
3  [[[0.14509804546833002, 0.15686275064945202, 0...  Kříženec  6 měsíců   
4  [[[0.7254902124404901, 0.623529434204101, 0.57...  Jezevčík     5 let   

          barva2             velikost2  \
0          Černá     Střední - 10-17kg   
1     

In [21]:
# Create a mapping of pet identifiers to integer indices
pet_ids = list(first_100.keys())
pet_to_idx = {pet_id: idx for idx, pet_id in enumerate(pet_ids)}

# Generate embeddings for all pets and store them
embeddings_dict = {
    pet_key: generate_embeddings(model, pet_info['images'])
    for pet_key, pet_info in first_100.items()
}

def generate_triplet(pet_key, pet_info, embeddings_dict, first_100):
    embeddings = embeddings_dict[pet_key]  # Use pre-generated embeddings

    # Create triplets: anchor, positive, and negative
    for i in range(len(embeddings)):
        anchor = embeddings[i]

        # Pick a random positive example from the same pet's observation, excluding the anchor image
        positive_idx = np.random.choice([idx for idx in range(len(embeddings)) if idx != i ])
        positive = embeddings[positive_idx]

        # Pick a random negative example (different pet)
        negative_pet_key = np.random.choice([key for key in first_100 if key != pet_key])
        negative_embedding = embeddings_dict[negative_pet_key][0]  # Use pre-generated negative embedding
       # print(i)
        print(positive_idx)
     #   print(negative_pet_key)
        yield [anchor, positive, negative_embedding], pet_to_idx[pet_key]
    #print(embeddings_dict[pet_key])

# Prepare the data for triplet training
triplets = []
labels = []

for pet_key, pet_info in first_100.items():
    for triplet, label in generate_triplet(pet_key, pet_info, embeddings_dict, first_100):
        triplets.append(triplet)
        labels.append(label)

# Convert triplets to numpy array
triplets = np.array(triplets)

# Flatten the embeddings for FAISS
flattened_triplets = np.array([
    np.concatenate([anchor.flatten(), positive.flatten(), negative.flatten()])
    for anchor, positive, negative in triplets
])



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


KeyboardInterrupt: 

In [87]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(flattened_triplets, labels, test_size=0.2, random_state=42)

# Initialize FAISS index for similarity search
embedding_dim = X_train.shape[1]  # The size of the flattened embeddings
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance metric for similarity search

# Add training data embeddings to the FAISS index
index.add(X_train)  # No need to reshape, as X_train is already flat

In [88]:
input_dim = 3840

# Create the model
model = create_model(input_dim)

# Compile and train the model
model.compile(optimizer='adam', loss=triplet_loss(margin=1.0))
model.fit(X_train, np.array(y_train), epochs=60, batch_size=32)

Epoch 1/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 1.0000
Epoch 2/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0000
Epoch 3/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0000
Epoch 4/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.0000
Epoch 5/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.0000
Epoch 6/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0000
Epoch 7/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0000
Epoch 8/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.0000
Epoch 9/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.0000
Epoch 10/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0000
Epoch 11/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.0000
Epoch 12/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.0000
Epoch 13/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0000
Epoch 14/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.0000
Epoch 15/60
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.0000
Epoc